In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from functools import partial

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params

from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

from tf_lab.iclr.helper import load_multiple_version_of_pcs, find_best_validation_epoch_from_train_stats

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
experiment_tag = 'two_branch_decoder'
loss = 'chamfer'
n_pc_points = 2048
random_seed = 42
class_name = raw_input('Give me the class type: ').lower()

Give me the class type: chair


In [5]:
syn_id = snc_category_to_synth_id()[class_name]
in_data = load_multiple_version_of_pcs('uniform_one' ,syn_id, n_classes=1)

679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [26]:
from tf_lab.point_clouds.encoders_decoders import decoder_with_convs_only, decoder_with_fc_only
from tf_lab.fundamentals.layers import conv_1d_tranpose


# layer = deconvolution_op(layer, activation='relu', nb_filter=64, filter_size=2, strides=1)
# avg_pool_1d()


def two_branch_decoder(layer, ae):
    l_e = tf.expand_dims(layer, 2)
    d0 = tf.nn.relu(conv_1d_tranpose(l_e, nb_filter=8, filter_size=8, strides=1))
    print d0
    d1 = tf.nn.relu(conv_1d_tranpose(d0, nb_filter=16, filter_size=4, strides=1))
    print d1
    conv_branch = conv_1d_tranpose(d1, nb_filter=24, filter_size=2, strides=1)
    conv_branch = tf.reshape(conv_branch, [-1, 1024, 3])
    ae.conv_branch = conv_branch
    
    print conv_branch
    fc_branch = decoder_with_fc_only(layer, [256, 256, 1024*3], b_norm=False)    
    fc_branch = tf.reshape(fc_branch, [-1, 1024, 3])
    ae.fc_branch = fc_branch
    return tf.concat([fc_branch, conv_branch], axis=1)

In [7]:
layer = tf.placeholder(tf.float32, [None, 128])
# two_branch_decoder(layer, )

In [9]:
bneck = 128

In [27]:
train_params = default_train_params()
experiment_id = '_'.join(['ae', class_name, experiment_tag, str(n_pc_points), 'pts', str(bneck), 'bneck', loss])
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/', experiment_id)
create_dir(train_dir)

reset_tf_graph()    
encoder, _, enc_args, _ = mlp_architecture_ala_iclr_18(n_pc_points, bneck)    

decoder = two_branch_decoder
dec_args = {'ae': self}

conf = Conf(n_input = [n_pc_points, 3],
            loss = loss,
            training_epochs = 500,
            batch_size = train_params['batch_size'],
            denoising = False,
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = train_params['saver_step'],
            z_rotate = False,
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
print conf
conf.experiment_name = 'experiment_' + str(experiment_id)
conf.held_out_step = 5
# conf.save(osp.join(train_dir, 'configuration'))
ae = PointNetAutoEncoder(conf.experiment_name, conf)
buf_size = 1 # flush each line
fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
#     train_stats = ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['val'])
train_stats = ae.train(in_data['train'], conf, log_file=fout, held_out_data=in_data['test'])
fout.close()

NameError: name 'self' is not defined

In [25]:
# g = ae.graph
# g.get_collection_ref('experiment_ae_chair_two_branch_decoder_2048_pts_128_bneck_chamfer/deconv_1d_2')

g.get_collection('trainable_variables')

# <tf.Variable 'experiment_ae_chair_two_branch_decoder_2048_pts_128_bneck_chamfer/deconv_1d_2/W:0' shape=(2, 24, 16) dtype=float32_ref>
# for op in g.get_operations():
#     print str(op.name) 

g.get_tensor_by_name('experiment_ae_chair_two_branch_decoder_2048_pts_128_bneck_chamfer/deconv_1d_2/W:0')

# # # get_operation_by_name('experiment_ae_chair_two_branch_decoder_2048_pts_128_bneck_chamfer/deconv_1d_2')

<tf.Tensor 'experiment_ae_chair_two_branch_decoder_2048_pts_128_bneck_chamfer/deconv_1d_2/W:0' shape=(2, 24, 16) dtype=float32_ref>